#### 뉴스 분류기 모델 생성

[1] 데이터 불러오기<hr>
- 정치:0, 경제:1, 사회:2, 생활문화:3, 세계:4, 기술:5, 연예:6, 스포츠:7
- 라벨당 200개의 기사


In [82]:
import pandas as pd
import numpy as np
import re
from NLPfunc import *

In [83]:
# 데이터 불러오기
# -> 데이터 : 라벨로 데이터프레임으로 만들기
# 각 폴더당 0~199까지
PATH='../data/news/'

In [84]:
# 각 텍스트에 라벨을 추가하여 하나의 데이터프레임으로 만듦
def makeDF(loc):
    PATH=loc
    newsDF=pd.DataFrame(columns=['text', 'label'])
    for j in range(8):
        text_list=[]
        dataDF=pd.DataFrame()
        for i in range(200):
            if i<10:
                path=PATH+f'{j}/{j}00{i}NewsData.txt'
            elif i<100:
                path=PATH+f'{j}/{j}0{i}NewsData.txt'
            else:
                path=PATH+f'{j}/{j}{i}NewsData.txt'
            with open(path, mode='r') as f:
                a= f.read()
                # 한글만 남겨놓고 제거
                a= re.sub('[^ㄱ-ㅎ가-힣]+',' ',a)
                text_list.append(a)
        dataDF['text']=text_list
        dataDF['label']=j
        if j==0:
            newsDF=dataDF.copy()
        else:
            newsDF=pd.concat([newsDF, dataDF], ignore_index=True)
    return newsDF

In [85]:
newsDF=makeDF(PATH)

In [86]:
newsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1600 non-null   object
 1   label   1600 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.1+ KB


[2] 데이터 전처리<hr>
- 토큰화 및 단어사전 생성
- 문장 벡터화 및 패딩

In [87]:
from konlpy.tag import Okt
from collections import Counter


In [88]:
# 학습, 테스트 분리 후 DS
trainDF=newsDF.sample(frac=0.9, random_state=1009).reset_index(drop=True)
testDF=newsDF.drop(trainDF.index).reset_index(drop=True)
trainDS= TextDataset(trainDF['text'], trainDF['label'])
testDS= TextDataset(testDF['text'], testDF['label'])
# 토큰화 인스턴스
okt=Okt()

In [89]:
## 불용어 리스트 불러오는 함수
STOP_PATH = 'kor_stopwords.txt'

def make_stopwords(STOP_PATH):
    with open(STOP_PATH, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines() # 문장단위로
    return set(stopwords)  

stopwords = make_stopwords(STOP_PATH)

{'ㅈ',
 '가',
 '가까스로',
 '가라',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '감',
 '갔나',
 '갖고말하자면',
 '같다',
 '같이',
 '개',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '걸',
 '것',
 '것과 같이',
 '것들',
 '게',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고',
 '고려하면',
 '고로',
 '곧',
 '곳',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까요',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '나왔는데',
 '남들',
 '남짓',
 '내',
 '냐',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '는',
 '니',
 '다',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

In [103]:
# 토큰화를 통한 단어사전 생성 ->Counter와 리스트 컴프리헨션을 통해
# - 행마다 데이터를 추출하여 토큰 반환 및 Counter에 저장
def make_vocab(data, tag, stopwords):
    '''
    데이터를 통해 단어사전을 만드는 함수 \n
    DF 데이터 분리했을 경우 인덱스 초기화하기!\n
    단어사전 반환\n
    params: 데이터프레임
    '''
    token_list=[]
    counter=Counter()
    for text in data:
        # 한글빼고 다지우기
        # text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)
        # 형태소 분석 (stem-> 어근으로 통일, norm-> 정규화)
        tokens=tag.morphs(text)
        token_list.append(tokens)
        #불용어, 구두점, 특수문자 제거

    # 형태소 단어 counter에 저장
    for t in token_list:
        for token in t:
            for s in stopwords:
                if t == s:  #불용어 제거
                    t.remove(token)
        counter.update(t)
    # 단어 사전에 저장
    vocab={'<PAD>':0, '<UNK>':1}
    vocab.update({word: i+2 for i, word in enumerate(counter.items())})
    return vocab

In [104]:
# 토큰화 및 단어사전 생성
train_vocab= make_vocab(data=trainDF['text'], tag=okt, stopwords=stopwords)
test_vocab= make_vocab(data=testDF['text'], tag=okt, stopwords=stopwords)

In [105]:
print(train_vocab)
print(len(train_vocab))

{'<PAD>': 0, '<UNK>': 1, ('노동시간', 7): 2, ('단축', 20): 3, ('최저임금', 46): 4, ('보다', 541): 5, ('경', 224): 6, ('제', 650): 7, ('파급', 7): 8, ('력', 141): 9, ('크다', 52): 10, ('앵커', 123): 11, ('하반기', 50): 12, ('부터', 753): 13, ('시행', 89): 14, ('되는', 460): 15, ('이', 14997): 16, ('인상', 111): 17, ('우리', 474): 18, ('경제', 235): 19, ('에', 11489): 20, ('더', 612): 21, ('큰', 340): 22, ('영향', 216): 23, ('을', 16413): 24, ('줄', 113): 25, ('것', 4332): 26, ('이란', 196): 27, ('주장', 387): 28, ('나오고', 52): 29, ('있습니다', 243): 30, ('정부', 550): 31, ('의', 10205): 32, ('지', 496): 33, ('원', 753): 34, ('방안', 123): 35, ('은', 8396): 36, ('일시', 32): 37, ('적', 2309): 38, ('인', 2100): 39, ('대책', 109): 40, ('일', 3285): 41, ('뿐이기', 1): 42, ('때문', 617): 43, ('연', 434): 44, ('착륙', 20): 45, ('위', 931): 46, ('한', 5576): 47, ('근본', 20): 48, ('마련', 131): 49, ('필요하다는', 20): 50, ('입니다', 444): 51, ('박진', 3): 52, ('형', 225): 53, ('기자', 1164): 54, ('보도', 232): 55, ('오는', 245): 56, ('월', 1376): 57, ('주', 465): 58, ('최대', 196): 59, ('시간', 77

In [106]:
for t in trainDF['text']:
    a=okt.morphs(t)
    print(a)
    break

['노동시간', '단축', '최저임금', '보다', '경', '제', '파급', '력', '크다', '노동시간', '단축', '최저임금', '보다', '경', '제', '파급', '력', '크다', '앵커', '하반기', '부터', '시행', '되는', '노동시간', '단축', '이', '최저임금', '인상', '보다', '우리', '경제', '에', '더', '큰', '영향', '을', '줄', '것', '이란', '주장', '이', '나오고', '있습니다', '정부', '의', '지', '원', '방안', '은', '일시', '적', '인', '대책', '일', '뿐이기', '때문', '에', '연', '착륙', '을', '위', '한', '근본', '적', '인', '대책', '마련', '이', '필요하다는', '것', '입니다', '박진', '형', '기자', '의', '보도', '입니다', '기자', '오는', '월', '부터', '주', '최대', '시간', '으로', '노동시간', '을', '줄이는', '제도', '가', '시행', '됩니다', '전문가', '들', '은', '최저임금제', '이상', '으로', '우리', '경제', '에', '큰', '영향', '을', '미칠', '것', '이라고', '입', '을', '모았습니다', '성태', '윤', '연세대', '경제학', '과', '교수', '최저임금', '의', '경우', '에는', '자영', '업자', '를', '중심', '으로', '가장', '큰', '타격', '이', '발생', '했다고', '볼', '수', '있고요', '근로시간', '단축', '의', '경우', '에는', '일반', '적', '인', '중소기업', '까지', '영향', '이', '확산', '될', '것', '으로', '보이', '고', '그렇지만', '우리', '경제', '의', '구조', '적', '문제', '해결', '을', '위해', '필요한', '제', '도란', '주장', '도', '있습니다', '정세', 

In [107]:
len(train_vocab)

34335

In [108]:
print(train_vocab)

{'<PAD>': 0, '<UNK>': 1, ('노동시간', 7): 2, ('단축', 20): 3, ('최저임금', 46): 4, ('보다', 541): 5, ('경', 224): 6, ('제', 650): 7, ('파급', 7): 8, ('력', 141): 9, ('크다', 52): 10, ('앵커', 123): 11, ('하반기', 50): 12, ('부터', 753): 13, ('시행', 89): 14, ('되는', 460): 15, ('이', 14997): 16, ('인상', 111): 17, ('우리', 474): 18, ('경제', 235): 19, ('에', 11489): 20, ('더', 612): 21, ('큰', 340): 22, ('영향', 216): 23, ('을', 16413): 24, ('줄', 113): 25, ('것', 4332): 26, ('이란', 196): 27, ('주장', 387): 28, ('나오고', 52): 29, ('있습니다', 243): 30, ('정부', 550): 31, ('의', 10205): 32, ('지', 496): 33, ('원', 753): 34, ('방안', 123): 35, ('은', 8396): 36, ('일시', 32): 37, ('적', 2309): 38, ('인', 2100): 39, ('대책', 109): 40, ('일', 3285): 41, ('뿐이기', 1): 42, ('때문', 617): 43, ('연', 434): 44, ('착륙', 20): 45, ('위', 931): 46, ('한', 5576): 47, ('근본', 20): 48, ('마련', 131): 49, ('필요하다는', 20): 50, ('입니다', 444): 51, ('박진', 3): 52, ('형', 225): 53, ('기자', 1164): 54, ('보도', 232): 55, ('오는', 245): 56, ('월', 1376): 57, ('주', 465): 58, ('최대', 196): 59, ('시간', 77

In [109]:
# vector_list=[]
# vecDF=trainDF.copy()
# for t in trainDF['text']:
#     token_lists=okt.morphs(t)
#     vector_token=[train_vocab[token] if token in train_vocab else train_vocab['<UNK>'] for token in token_lists]
#     vector_list.append(vector_token)
# print(len(vector_list))
# vecDF['text']=vector_list


In [110]:
def vectorize(vocab, DF, tokenizer):
    '''
    단어사전을 통해 문장을 수치화하는 함수

    '''
    vector_list=[]
    vecDF=DF.copy()
    for t in DF['text']:
        token_lists=tokenizer.morphs(t)
        vector_token=[vocab[token] if token in vocab else vocab['<UNK>'] for token in token_lists]
        vector_list.append(vector_token)
    vecDF['text']=vector_list

    return vecDF

In [111]:
# 문장 벡터화
trainVec=vectorize(vocab=train_vocab, DF=trainDF, tokenizer=okt)
testVec=vectorize(vocab=test_vocab, DF=testDF, tokenizer=okt)

In [112]:
# 패딩
def padding(length, textList):
    pad_texts=[]
    for text in textList:
            # 선택 길이> 문장길이 일때
        if length>len(text):
                                            # 남은 텍스트 길이만큼 0으로 채우기
            text=text+[0]*(length-len(text))
        else: # 선택 길이< 문장 길이 일때
            text=text[:length]              # 선택 길이만큼 슬라이싱
        pad_texts.append(text)
    return pad_texts

In [113]:
train_pad=padding(length=500, textList=trainVec['text'])
trainPad=trainVec.copy()
trainPad['text']=train_pad
test_pad= padding(length=500, textList=testVec['text'])
testPad=testVec.copy()
testPad['text']=test_pad

In [114]:
# 데이터셋, 데이터로더 생성
import torch

train_tensor=torch.tensor(train_pad)
train_label= torch.FloatTensor(trainVec['label'].values)

test_tensor=torch.tensor(test_pad)
test_label= torch.FloatTensor(testVec['label'].values)

trainDS=TextDataset(train_tensor, train_label)
testDS= TextDataset(test_tensor, test_label)

trainDL=DataLoader(trainDS, batch_size=20)
testDL=DataLoader(testDS, batch_size=20)


In [115]:
train_label

tensor([1., 7., 7.,  ..., 6., 6., 2.])

In [116]:
# 모델 흐름
# 1. 입력받은 값을 통해 임베딩 값 얻음
# 2. 임베딩 값을 통해 출력값 얻음
# 3. 출력값의 마지막 시점만을 활용하여 분류
import torch.nn as nn
from typing import Literal


class textCLF(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers,
                 model_type: Literal['lstm', 'rnn'], dropout=0.5, bidirectional=True):
        super().__init__()

        # 임베딩 층
        # num-> 단어사전의 크기
        self.embedding= nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type =='rnn':
            self.model=nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True
            )
        elif model_type =='lstm':
            self.model= nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True
            )
        # 양방향 여부
        if bidirectional:
            self.classifier= nn.Linear(hidden_dim*2, 8)
        else:
            self.classifier= nn.Linear(hidden_dim, 8)
        self.dropout=nn.Dropout(dropout)

    #포워딩
    def forward(self, input):
        embeddings= self.embedding(input)
        output,_= self.model(embeddings)
        last_output=output[:,-1,:]
        last_output=self.dropout(last_output)
        logits=self.classifier(last_output)
        return logits

In [117]:
from torch import optim
# 파라미터 설정

n_vocab= len(train_vocab)
hidden_dim= 64
embedding_dim= 128
n_layers= 2

device= 'cuda' if torch.cuda.is_available() else 'cpu'
classifier= textCLF(
    n_vocab=n_vocab, hidden_dim=hidden_dim, 
    embedding_dim=embedding_dim, n_layers=n_layers, model_type='lstm'
).to(device)
criterion= nn.CrossEntropyLoss()

optimizer= optim.Adam(classifier.parameters(), lr=0.001)

In [118]:
from get_model import Custom_model
from torchmetrics.classification import F1Score, MulticlassF1Score
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.regression import R2Score, MeanSquaredError
import torch
import matplotlib.pyplot as plt
from typing import Literal
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler

def model_training(model, trainDL, testDL, optimizer, epoch: int, LIMIT: int, break_param: Literal['score', 'loss'],
                   type: Literal['reg', 'binary', 'muticlass'],optim_type: Literal['score', 'loss'], SAVE_PATH, SAVE_FILE,
                   save_type: Literal['all', 'param', 'None'], numcls: int):
    '''
    학습진행+ 모니터링+ 최적의 결과 저장
    type= 'reg'|'binary'|'mclf'
    return: LOSS_HISTORY, SCORE_HISTORY
    '''
    scheduler= lr_scheduler.ReduceLROnPlateau(optimizer, patience=LIMIT, mode='max')
    EPOCH=epoch
    # 손실, 평가값 저장
    LOSS_HISTORY, SCORE_HISTORY= [[],[]], [[],[]]
    for ep in range(EPOCH):
        print(f'{ep+1}/{EPOCH}')
        model.train()
        loss_total, score_total= 0,0
        loss_val_total, score_val_total=0,0

        for train_feature, train_target in trainDL:
            # 학습
            pre_y=model(train_feature)

            # 손실
            if type=='reg':
                Lossfunc=MeanSquaredError()
                Scorefunc=R2Score()
            elif type=='binary':
                Lossfunc= nn.BCELoss()
                Scorefunc=F1Score(task='binary', num_classes=numcls)
            elif type=='muticlass':
                Lossfunc=nn.CrossEntropyLoss()
                Scorefunc=F1Score(task='multiclass', num_classes=numcls)
            loss= Lossfunc(pre_y, train_target.reshape(-1).long())
        
            loss_total+=loss.item()

            # 평가
            score= Scorefunc(pre_y, train_target.reshape(-1).long())
            score_total+=score.item()
            # 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 검증

        model.eval()
        with torch.no_grad():
            for val_feature, val_target in testDL:
                # 학습
                pre_val= model(val_feature)

                # 손실
                loss= Lossfunc(pre_val, val_target.reshape(-1).long() if type=='muticlass' else val_target if type=='reg' else val_target)
                loss_val_total+=loss.item()

                # 평가
                score= Scorefunc(pre_val, val_target.reshape(-1) if type=='muticlass' else val_target)
                score_val_total+=score.item()

        
        # 저장
        LOSS_HISTORY[0].append(loss_total/len(trainDL))
        SCORE_HISTORY[0].append(score_total/len(trainDL))
        print(f'Train\n Loss: {loss_total/len(trainDL)}\n Score: {score_total/len(trainDL)}')

        LOSS_HISTORY[1].append(loss_val_total/len(testDL))
        SCORE_HISTORY[1].append(score_val_total/len(testDL))
        print(f'Val\n Loss: {loss_val_total/len(testDL)}\n Score: {score_val_total/len(testDL)}')

        # 성능이 좋은 학습 가중치 저장
        # if save_type:
        #     if save_type=='all':
        #         save_type= model
        #     elif save_type=='param':
        #         save_type=model.state_dict()
        #     if len(SCORE_HISTORY[1]) == 1: 
        #     #첫번째는 무조건 저장
        #         torch.save(save_type, SAVE_PATH+SAVE_FILE)  
                
        #     else:
        #         if SCORE_HISTORY[1][-1]> max(SCORE_HISTORY[1][:-1]): # 자신을 제외한 최대점수값과 비교
        #             torch.save(save_type, SAVE_PATH+SAVE_FILE) 
                     
        # else: pass

        
        # 학습 진행 모니터링 (검증 데이터 개선이 되지 않았을때 누적 ->  평가, 손실 중 지표 하나 선택)
        # 최적화 스케쥴러 인스턴스 업데이트
        scheduler.step(score_val_total/len(testDL))
        # print(f'scheduler.num_bad_epochs: {scheduler.num_bad_epochs}', end=' ') #보여주기용
        # print(f'scheduler.patience: {scheduler.patience}')
        # 손실 감소 (또는 성능 개선)이 안되는 경우 조기종료
        if scheduler.num_bad_epochs== scheduler.patience:
            print(f'{scheduler.patience} EPOCH 성능 개선이 없어서 조기종료함')
            break

    return LOSS_HISTORY, SCORE_HISTORY

In [119]:
# from get_train_model import model_training

In [120]:
LOSS_HISTORY, SCORE_HISTORY= model_training(model= classifier, trainDL=trainDL,
                                            testDL=testDL, optimizer=optimizer,
                                            epoch=50, LIMIT=5, SAVE_PATH=None,
                                            SAVE_FILE=None, numcls=8,
                                            break_param='score', save_type='all',
                                            type='muticlass', optim_type='score')

1/50
Train
 Loss: 2.059150132868025
 Score: 0.15555555828743511
Val
 Loss: 1.829165980219841
 Score: 0.9562499895691872
2/50
Train
 Loss: 2.0459820297029285
 Score: 0.16666666925367382
Val
 Loss: 1.8486521542072296
 Score: 0.9562499895691872
3/50
Train
 Loss: 2.039994011322657
 Score: 0.16875000334241325
Val
 Loss: 1.8027099668979645
 Score: 0.9437499940395355
4/50
Train
 Loss: 2.0390626655684576
 Score: 0.17500000353902578
Val
 Loss: 1.8597099035978317
 Score: 0.9437499940395355
5/50
Train
 Loss: 2.034953036242061
 Score: 0.16111111444317633
Val
 Loss: 1.8480080962181091
 Score: 0.9187500029802322
6/50
Train
 Loss: 2.030292221241527
 Score: 0.17361111493988168
Val
 Loss: 1.8596364259719849
 Score: 0.9375
5 EPOCH 성능 개선이 없어서 조기종료함
